# Exercise 9.1: The Wave Equation in Cylindrical Geometry

The wave equation in cylindrical geometry often leads to the eigenvalue problem:

$\left( \frac{ \mathrm{d}^2 } { \mathrm{d} r^2 } + \frac{1}{r} \frac{ \mathrm{d}}{\mathrm{d} r} \right) \Phi(r) = -k^2 \Phi(r)$,

with initial conditions $\Phi(r=0) = 1$ and $\Phi(r=1) = 0$. 

The analytical eigenfunctions are the regular cylindrical Bessel functions of order zero, and the eigenvalues correspond to the zeros of this function: 

$k_1 = 2.404826$, $k_2 = 5.520078$, $k_3 = 8.653728$, $k_4 = 11.791534$, ...

(a) Show that the substitution $\Phi = r^{-1/2} \phi$ changes this equation into one for which the Numerov algorithm is appropriate. 

(b) Modify the code given in Example 9.7 to solve this problem. Compare the numerical eigenvalues with the exact values. Note that you will not be able to start the integration at $r=0$. Try starting at e.g. $r=10^{-13}$, with integration step size $h=10^{-5}$. 

Note that you should expect $\mathcal{O}(\%)$ precision, but not better.

## Solution (a)

Consider the substitution $\Phi\to r^{-1/2}\phi$. Then the derivatives become
$$\Phi'(r)=-\frac{1}{2}r^{-3/2}\phi(r)+r^{-1/2}\phi'(r)\hspace{.5cm}\Phi''(r)=\frac{3}{4}r^{-5/2}\phi(r)-r^{-3/2}\phi'(r)+r^{-1/2}\phi''(r),$$
and thus the wave equation is
$$\bigg[\frac{3}{4}r^{-5/2}-r^{-3/2}\frac{d}{dr}+r^{-1/2}\frac{d^2}{dr^2}+\frac{1}{r}\bigg(-\frac{1}{2}r^{-3/2}+r^{-1/2}\frac{d}{dr}\bigg)\bigg]\phi(r)=\bigg(\frac{1}{4}r^{-5/2}+r^{-1/2}\frac{d^2}{dr^2}\bigg)\phi(r)=-k^2r^{-1/2}\phi(r)$$
$$\implies r^{-1/2}\phi''(r)+\bigg(\frac{1}{4}r^{-5/2}+k^2r^{-1/2}\bigg)\phi(r)=0\implies\boxed{\phi''(r)+\bigg(\frac{1}{4}r^{-2}+k^2\bigg)\phi(r)=0}$$

## Solution (b)

Below we define a root-finder function ```bisect``` and an implementation of the Numerov algorithm ```Numerov```.

In [4]:
import numpy as np
from functools import partial

In [2]:
def bisect(func,a,b,prec,N): # searches for a root of func on the interval [a,b] via the midpoint method, and returns the root within precision prec; runs through N iterations
    left = a # left end of interval for which the root is searched; initially a
    right = b # right end of interval for which the root is searched; initially b
    mid = (a+b)/2 # the midpoint of the interval for which the root is searched; initially set as the midpoint of [a,b]
    count = 0 # counts the number of iterations
    while abs(func(mid)) > prec and count < N:
        if np.sign(func(left)) != np.sign(func(mid)): # chooses a new interval to search according to where the function values change sign
            left = left
            right = mid
            mid = (left+right)/2
        else:
            left = mid
            right = right
            mid = (left+right)/2
        count = count + 1
    return mid

# Numerov's algorithm (forward)
# takes as input the initial conditions y(0) and y'(0) as y0 and yp0
# h is the step size, the k-squared term (k2), the S term -- these are FUNCTIONS!
# the initial value of the independent variable x0, and the final value xf
# returns t,y as the solution arrays
def Numerov(k2, S, y0, yp0, h, x0, xf):
    """Returns the solution to a 2nd-order ODEs of the type: y'' + k^2 y = S(x) via the Numerov algorithm"""
    # the number of steps:
    N = int( (xf-x0)/h ) # needs to be an integer
    # get y1 via Taylor series:
    y1 = y0 + h * yp0
    # define the numpy arrays to return
    ya = np.zeros(N+1)
    xa = np.zeros(N+1)
    # set the first two values of the arrays:
    ya[0] = y0
    ya[1] = y1
    xa[0] = x0
    xa[1] = x0 + h
    # integrate via the Numerov algo:
    for n in range(1,N):
        x = x0 + n*h
        xa[n]=x
        h2dt = h**2/12 # appears often so let's just calculate it once!
        ya[n+1] = (2 * (1 - 5*h2dt * k2(x)) * ya[n] - (1 + h2dt *k2(x-h)) * ya[n-1] + h2dt*(S(x+h) + 10 * S(x) + S(x-h)))/((1 + h2dt * k2(x+h) ))    
    xa[N] = xf # set the last x value which is not set in the loop
    return xa,ya

We want use ```numerov``` to solve the equation in part (a) for some initial guess of the parameter $k$. Let us implement the solution below.

In [15]:
def k2(k,r): # the function multiplying phi
    return 1/4*r**(-2)+k**2
def s(x): # the source function
    return 0
    
def trial(k): # solves the equation in part (a) for a particular k, with initial conditions phi(1E-13)=1 and phi'(1E-13)=1, step size 1E-3, and propagates to 1
    trl = Numerov(partial(k2,k),s,1,1,1E-3,1E-13,1) 
    return trl

The idea is to treat the value of the value of the solution to part (a) at the boundary of the cylinder as a function of $k$, and to then use a simple root finder to deduce the eigenvalues. This is done below.

In [16]:
def phi1(k): # returns the Numerov solution at r=1
    return trial(k)[1][-1]

aList = [2,5,8,11] # list of bisection left enpoints
bList = [3,6,9,11] # list of bisection right endpoints

for i in range(len(aList)): 
        root = bisect(phi1,aList[i],bList[i],1E-2,1000) # func,a,b,prec,N
        print("Eigenvalue", i+1, ":", root)

Eigenvalue 1 : 2.657606705605394
Eigenvalue 2 : 5.814967488304065
Eigenvalue 3 : 8.976839209493946
Eigenvalue 4 : 11.0
